## 1. Boundary box on video

In [2]:
import numpy as np
import cv2

#/Users/pawankumarkc/Downloads/Datasets_to_delete/CV/Video

In [3]:
body_classifier = cv2.CascadeClassifier('/Users/pawankumarkc/Downloads/Datasets_to_delete/Haarcascades/haarcascade_fullbody.xml')
cap = cv2.VideoCapture('/Users/pawankumarkc/Downloads/Datasets_to_delete/CV/Video/vtest.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bodies = body_classifier.detectMultiScale(grey, 1.2, 3)

    for (x, y, w, h) in bodies:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255),2 )
        cv2.imshow('Pedestrians', frame)

    if cv2.waitKey(1) ==13:
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


## 2. Count cars on the road - Video count 

In [3]:
from time import sleep

In [11]:
min_width_rec = 80  
min_height_rec = 80

offset=6    # Allow error between pixel
delay=60    # FPS to video
carros=0    #Counter - No. of detected vehicles
count_lines_pos = 550
detect =[]

def central_handle(x, y, w, h):
    #Calculate centre coordinate of a rectangle
    x1 = int(w/2)
    y1 = int(w/2)
    cx = x + x1
    cy = y + y1
    return cx, cy


car_cap = cv2.VideoCapture('/Users/pawankumarkc/Downloads/Datasets_to_delete/CV/Video/video.mp4')
algo = cv2.bgsegm.createBackgroundSubtractorMOG()
#cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = car_cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3), 5)

    #applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)
    counterShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cv2.line(img=frame, pt1=(25, count_lines_pos), pt2=(1200, count_lines_pos), color=(0,0,255), thickness=3)
    for (i, c) in enumerate(counterShape):
        x, y, w, h = cv2.boundingRect(c)
        valid_contorno = (w >= min_width_rec) and (h >= min_height_rec)

        if not valid_contorno:
            continue

        cv2.rectangle(img=frame, pt1=(x,y), pt2=(x+w, y+h), color=(0,255,0), thickness=2)
        cv2.putText(img=frame, text=f'Vehicle {carros}', org=(x, y-20), 
                    fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(255,244,0), thickness=2)
        
        center = central_handle(x, y, w, h)
        detect.append(center)
        cv2.circle(img=frame, center=center, radius=4, color=(255,100,100), thickness=-1)

        for (x, y) in detect:
            if y <  (count_lines_pos + offset) and y > (count_lines_pos - offset):
                carros +=1 
                cv2.line(img=frame, pt1=(25, count_lines_pos), pt2=(1200, count_lines_pos), color=(0,0,255), thickness=3)
                detect.remove((x, y))
                print('Car is dected :'+str(carros))
    
    cv2.putText(frame, 'Vehicle count : '+str(carros), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,255), 5)


    cv2.imshow('Original video with vehicle count', frame)

    if cv2.waitKey(1) == 13:
        break

car_cap.release()
cv2.destroyAllWindows()


Car is dected :1
Car is dected :2
Car is dected :3
Car is dected :4
Car is dected :5
Car is dected :6
Car is dected :7
Car is dected :8
Car is dected :9
Car is dected :10
Car is dected :11
Car is dected :12
Car is dected :13
Car is dected :14
Car is dected :15
Car is dected :16
Car is dected :17
Car is dected :18
Car is dected :19
Car is dected :20
Car is dected :21
Car is dected :22
Car is dected :23
Car is dected :24
Car is dected :25
Car is dected :26
Car is dected :27
Car is dected :28
Car is dected :29
Car is dected :30
Car is dected :31
Car is dected :32
Car is dected :33
Car is dected :34
Car is dected :35
Car is dected :36
Car is dected :37
Car is dected :38
Car is dected :39
Car is dected :40
Car is dected :41


## 3. Hand Gesture

In [1]:
import cv2
import numpy as np
import time
import mediapipe as mp

In [2]:
cap = cv2.VideoCapture(0) # 0 = Live video
mpHands = mp.solutions.hands
hands = mpHands.Hands()

mpDraw = mp.solutions.drawing_utils

#Frame per sec
pTime = 0
cTime = 0 

while True:
    ret, img = cap.read()
    imgBgr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resluts = hands.process(imgBgr)

    print(resluts.multi_hand_landmarks)
    cv2.imshow('Hand image', img)

    if resluts.multi_hand_landmarks:
        for handlms in resluts.multi_hand_landmarks:
            for id, lm in enumerate(handlms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                print(id, cx, cy)

                if id==4:
                    cv2.circle(img, (cx,cy), 15, (255,0,255), cv2.FILLED)
            
            mpDraw.draw_landmarks(img, handlms, mpHands.HAND_CONNECTIONS)
        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)
        cv2.imshow('Image', img)

        if cv2.waitKey(1)==13:
            break

cap.release()
cv2.destroyAllWindows()

2023-11-26 12:28:13.691 Python[5565:114879] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1700981894.679791       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


2023-11-26 12:28:37.397 Python[5565:114879] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.14474936
  y: 0.9734038
  z: 9.5918416e-08
}
landmark {
  x: 0.14136256
  y: 0.9304632
  z: 0.001022953
}
landmark {
  x: 0.13613862
  y: 0.8897833
  z: -0.0001245039
}
landmar

error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
